Lessons Learnt
1)How to use predefiend tool
2)How to make a simple tool of yourself
3)How to make a tool which is dependent on your datasets,vectorbases etc which you want to use more complexly

In [ ]:
from dotenv import load_dotenv
import os
import pickle
load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [ ]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

In [ ]:
from langchain.agents import load_tools
from langchain.llms import OpenAI
llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY)
tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

In [ ]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Math Tool",
        func=tools[0].run,
        description="Tool to calculate, nothing else"
    )
]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)
agent.run("How are you?")

In [ ]:
agent.run("What is 100 devided by 25?")

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:

agent.run("When does the restaurant open?") 